[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mlrghv/fastai/blob/master/cnn.ipynb)

In [0]:
??Conv2D

In [0]:
import keras
from keras.models import Model
from keras.layers import Input, Dense
import matplotlib.pyplot as plt
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Bidirectional, MaxPooling1D, MaxPooling2D, Reshape, Flatten, concatenate, BatchNormalization
from keras.layers import Conv2D
from keras.utils import plot_model
import tensorflow as tf
import numpy as np
from keras.callbacks import EarlyStopping
from keras.datasets import mnist
import argparse
from keras.models import load_model


In [0]:
from keras.datasets import mnist
def getData(nb_classes=10, img_rows=28, img_cols=28):
  (X_train, y_train), (X_test, y_test) = mnist.load_data()

  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train /= 255
  X_test /= 255

  # one-hot encoding:
  Y_train =  keras.utils.to_categorical(y_train, nb_classes)
  Y_test =  keras.utils.to_categorical(y_test, nb_classes)

  X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
  X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)
  return X_train,Y_train,X_test,Y_test

In [0]:
def cnn_model():
  sequence_input = Input(shape=(28,28,1), dtype='float64')
  
  x1 = Conv2D(32,kernel_size=5, strides=(1,1), padding = 'same' ,activation='relu')(sequence_input)
  x1_1 = MaxPooling2D(2)(x1)
  
  x2_1 = Conv2D(64,kernel_size=3,strides=(1,1),padding='same',activation='relu')(x1_1)
  x2_2 = Conv2D(64,kernel_size=3,strides=(1,1),padding='same',activation='relu')(x1_1)
  x2_1 = Dropout(0.3)(x2_1)
  x2_2 = Dropout(0.3)(x2_2)

  x3_1 = Conv2D(256,kernel_size=3,strides=2,padding='same',activation='relu')(x2_1)
  x3_2 = Conv2D(256,kernel_size=3,strides=2,padding='same',activation='relu')(x2_2)

  x4 = concatenate([x3_1, x3_2])
  x4 = Dropout(0.3)(x4)
  x4 = MaxPooling2D(2)(x4)
  x4 = Flatten()(x4)
  
  x5 = Dense(1000, activation='relu')(x4)
  
  x6 = Dense(500, activation='relu')(x5)
  
  output = Dense(10, activation='softmax')(x6)
  model = Model(sequence_input, output)
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [0]:
def cnn_model_test():
  sequence_input = Input(shape=(28,28,1), dtype='float64')
  
  x1 = Conv2D(32,1,activation='relu')(sequence_input)
  x1_1 = MaxPooling2D(2)(x1)
  
  x2_1 = Conv2D(64,1,activation='relu')(x1_1)
  x2_2 = Conv2D(64,1,activation='relu')(x1_1)
  x2_1 = Dropout(0.3)(x2_1)
  x2_2 = Dropout(0.3)(x2_2)

  x3_1 = Conv2D(256,2,strides=2,activation='relu')(x2_1)
  x3_2 = Conv2D(256,2,strides=2,activation='relu')(x2_2)

  x4 = concatenate([x3_1, x3_2])
  x4 = Dropout(0.3)(x4)
  x4 = MaxPooling2D(2)(x4)
  x4 = Flatten()(x4)
  
  x5 = Dense(1000, activation='relu')(x4)
  
  x6 = Dense(500, activation='relu')(x5)
  
  output = Dense(10, activation='softmax')(x6)
  model = Model(sequence_input, output)
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [0]:
def accuracy(test_x, test_y, model):
  result = model.predict(test_x)
  predicted_class = np.argmax(result, axis=1)
  true_class = np.argmax(test_y, axis=1)
  num_correct = np.sum(predicted_class == true_class) 
  accuracy = float(num_correct)/result.shape[0]
  return (accuracy * 100)

In [0]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='MNIST')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=20, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--optimizer', type=str, default='adam', metavar='O',
                        help='Optimizer (default: adam)')
    args = parser.parse_args()
    x_train,y_train,x_test,y_test = getData()
    x_train,y_train,x_test,y_test = preprocess(x_train,y_train,x_test,y_test)
    train(args)

In [0]:
x_train,y_train,x_test,y_test = getData()
#x_train,y_train,x_test,y_test = preprocess(x_train,y_train,x_test,y_test)
train(None)


In [0]:
def train(args):
  model = cnn_model_test()
  earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
  callbacks_list = [earlystop]
  model.fit(x_train,y_train,epochs=10, batch_size=128,verbose=2, validation_split=0.2, callbacks=callbacks_list)
  model.save('my_model.h5')


In [0]:
train(None)
accuracy(x_test,y_test,load_model('my_model.h5'))

In [0]:
def predict_image(img):
  # Preprocess the image so that it matches the training input
  image = Image.open(img)
  image = np.asarray(image.resize((28,28)))
  image = image.reshape(1,1,28,28)

  # Use the loaded model to generate a prediction.
  pred = model.predict(image)

  # Prepare and send the response.
  digit = np.argmax(pred)
  prediction = {'digit':int(digit)}
  return jsonify(prediction)

In [0]:
if __name__ == "__main__":
  main()

In [0]:
sequence_input = Input(shape=(28,28,1), dtype='float64')
x1 = Conv2D(32,kernel_size=5, strides=(1,1), padding = 'same' ,activation='relu')(sequence_input)
x1_1 = MaxPooling2D(2)(x1)

x2_1 = Conv2D(64,kernel_size=3,strides=(1,1),padding='same',activation='relu')(x1_1)
x2_2 = Conv2D(64,kernel_size=3,strides=(1,1),padding='same',activation='relu')(x1_1)
x2_1 = Dropout(0.3)(x2_1)
x2_2 = Dropout(0.3)(x2_2)

x3_1 = Conv2D(256,kernel_size=3,strides=2,padding='same',activation='relu')(x2_1)
x3_2 = Conv2D(256,kernel_size=3,strides=2,padding='same',activation='relu')(x2_2)

x4 = concatenate([x3_1, x3_2])
x4 = Dropout(0.3)(x4)
x4 = MaxPooling2D(2)(x4)
x4 = Flatten()(x4)

x5 = Dense(1000, activation='relu')(x4)

x6 = Dense(500, activation='relu')(x5)

output = Dense(10, activation='softmax')(x6)
model = Model(sequence_input, output)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])


In [0]:
model.summary()

In [0]:
model = cnn_model_test()
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                        verbose=1, mode='auto')
callbacks_list = [earlystop]
model.fit(x_train,y_train,epochs=10, batch_size=128,verbose=2, validation_split=0.2, callbacks=callbacks_list)


In [0]:
accuracy(x_test,y_test,model)

